In [74]:
from collections import defaultdict
from glob import glob
from itertools import chain
from json import load
from operator import itemgetter

from graphviz import Graph, Digraph

In [75]:
ENG = 'dot'

In [76]:
A = "Q:/Downloads/bn/current/data/json/mutations/mutations.json"

with open(A) as F:
    data = load(F)

name_lookup = {
    datum['id']: datum['name']
    for datum in data
}

data = [
    {
        key: [
            name_lookup[id_]['str']
            for id_ in value
            if id_ not in "ROBUST|PAINREC2|PAINREC3".split("|")
        ] if key in ('changes_to', 'leads_to', 'prereqs', 'prereqs2', 'cancels', 'threshreq') else value
        for key, value in datum.items()
    }
    for datum in data
]

categories = {*chain.from_iterable(
    frozenset(datum.get('category', {}))
    for datum in data
)}

In [77]:
data_lookup = {
    (
        datum['name']['str']
        if isinstance(datum['name'], dict)
        else datum['name']
    ): datum
    for datum in data
}

In [78]:
def recursive_solver_1(mut):
    if 'depth' in mut:
        return mut['depth']
    depths = [
        recursive_solver_1(data_lookup[sub_mut])
        for sub_mut in mut.get('changes_to', []) + mut.get('leads_to', [])
        # if not print(sub_mut)
    ]
    mut['depth'] = depths and 1 + max(depths) or 0
    return mut['depth']

for datum in data:
    recursive_solver_1(datum)

data.sort(key=itemgetter('depth'), reverse=True)

In [79]:
def is_not_valid(datum):
    return mut.get('profession', False) or not mut.get('player_display', True) or mut.get('debug', False) or mut.get('threshold', False)

In [80]:
main_graph = Digraph('Mutations',
                     format='svg',
                     engine=ENG,
                     graph_attr={
                         'concentrate': 'false',
                     })

In [81]:
mutation_colors = {
    'LIZARD': 'khaki',
    'BIRD': 'deepskyblue',
    'FISH': 'aquamarine',
    'BEAST': 'tomato',
    'FELINE': 'sandybrown',
    'LUPINE': '#A9A9A9',
    'URSINE': 'brown',
    'CATTLE': 'peru',
    'INSECT': 'yellow',
    'PLANT': 'forestgreen',
    'SLIME': 'greenyellow',
    'TROGLOBITE': 'slategray',
    'CEPHALOPOD': 'purple',
    'SPIDER': 'orange',
    'RAT': 'grey',
    'MEDICAL': 'red',
    'ALPHA': 'skyblue',
    'ELFA': '#90EE90',
    'CHIMERA': 'indianred',
    'RAPTOR': 'green',
    'MOUSE': 'lightgrey',
    'MYCUS': 'violet',
    'MARLOSS': 'cyan',
    'NEUTRAL': '#FFFFFF'
}

mutation_category_colors = {}

for mut in data:
    cats = mut.get('category', [])
    name = isinstance(mut['name'], dict) and mut['name']['str'] or mut['name']
    if not cats:
        mutation_category_colors[name] = mutation_colors['NEUTRAL']
    else:
        mutation_category_colors[name] = ':'.join(mutation_colors[cat] for cat in sorted(cats))

In [82]:
category_clusters = {
    cat: Digraph(name=f'{cat}',
                 engine=ENG,
                 graph_attr={
                     'label': cat
                 },
                 node_attr={
                     'shape': 'rectangle',
                 })
    for cat in categories | {'NEUTRAL'}
}

for mut in data:
    this_name = isinstance(mut['name'], dict) and mut['name']['str'] or mut['name']
    if is_not_valid(mut):
        continue
    
    for cat in mut.get('category', ['NEUTRAL']):
        points = mut.get('points', 0)
        category_clusters[cat].node(this_name,
                                    shape='rectangle',
                                    style=','.join(filter(None, 
                                                          ['striped',
                                                           'dotted' if points < 0 else None])),
                                    fontsize='28' if mut.get('threshreq', []) else '14',
                                    fillcolor=mutation_category_colors[this_name])

In [83]:
for mut in data:
    this_name = isinstance(mut['name'], dict) and mut['name']['str'] or mut['name']
    if is_not_valid(mut):
        continue

    for name in mut.get('prereqs2', []):
        main_graph.edge(name, this_name, style='dotted', color='violet')

    for name in mut.get('changes_to', []):
        main_graph.edge(this_name, name, arrowhead='normal', color='black')
        # this_name in data_lookup[name].get('prereqs', []) and data_lookup[name]['prereqs'].remove(this_name)
        # this_name in data_lookup[name].get('prereqs2', []) and data_lookup[name]['prereqs2'].remove(this_name)

    for name in mut.get('leads_to', []):
        main_graph.edge(this_name, name, arrowhead='box', color='green')
        # this_name in data_lookup[name].get('prereqs', []) and data_lookup[name]['prereqs'].remove(this_name)
        # this_name in data_lookup[name].get('prereqs2', []) and data_lookup[name]['prereqs2'].remove(this_name)

    for name in mut.get('cancels', []):
        main_graph.edge(this_name, name, dir='none', color='red')
        this_name in data_lookup[name].get('cancels', []) and data_lookup[name]['cancels'].remove(this_name)

In [84]:
for cluster in category_clusters.values():
    main_graph.subgraph(cluster)

In [85]:
main_graph.render()
main_graph.format = 'png'
main_graph.render()

'Mutations.gv.png'

In [86]:
legend = Digraph(name='Legend', format='svg', engine='circo', node_attr={'shape': 'rectangle', 'style': 'striped'})


legend.node('Positive Regular Mutation', fontsize='14', fillcolor='#FFFFFF')
legend.node('Negative Regular Mutation', style='striped,dotted', fontsize='14', fillcolor='#FFFFFF')
legend.node('Positive Post-Threshold Mutation', fontsize='28')
legend.node('Positive Slime Post-Threshold Mutation', fontsize='28', fillcolor=mutation_colors['SLIME'])
legend.node('Regular Mutation of Slime and Chimera', fontsize='14', fillcolor=f'{mutation_colors["SLIME"]}:{mutation_colors["CHIMERA"]}')

legend.edge('Negative Regular Mutation', 'Positive Regular Mutation', arrowhead='normal', color='black', label='Changes to')
legend.edge('Positive Regular Mutation', 'Positive Post-Threshold Mutation', arrowhead='box', color='green', label='Leads to')
legend.edge('Positive Post-Threshold Mutation', 'Positive Slime Post-Threshold Mutation', style='dotted', color='blue', label='Requires')
legend.edge('Positive Regular Mutation', 'Positive Slime Post-Threshold Mutation', style='dotted', color='violet', label='Additional Requirement')
legend.edge('Negative Regular Mutation', 'Positive Slime Post-Threshold Mutation', dir='none', color='red', label='Cancels')

legend.node('Category Legend', shape='plaintext')
for cat, color in mutation_colors.items():
    legend.node(cat, fillcolor=color, style='filled', shape='diamond')
    legend.edge('Category Legend', cat, arrowhead='none', style='dashed')
    
legend.render()
legend.format = 'png'
legend.render()

'Legend.gv.png'